In [142]:
import os
from pathlib import Path

from dotenv import load_dotenv
from langchain_community.document_loaders import (
    UnstructuredEPubLoader,
    UnstructuredMarkdownLoader,
)
from llama_index.core import Document
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.notebook_utils import display_source_node

from readai.components.loaders import MarkdownReader

# 加载.env文件
load_dotenv()

# 获取项目根目录路径
project_root = Path(os.getenv("PROJECT_ROOT"))


In [148]:
test_data_path = project_root / "readai/tests/data"

### 使用epubloader，效果不太理想

In [147]:
# 第一步：使用 UnstructuredEPubLoader 加载 EPUB 文档
# 读取PROJECT_ROOT

book_name = "非暴力沟通.epub"
book_path = test_data_path / book_name
loader = UnstructuredEPubLoader(book_path, mode="elements", strategy="fast")
documents = loader.load()
print(f"节点数量: {len(documents)}")

节点数量: 1795


### epub转markdown
- 优点是处理起来更好更方便，可以在load之前还进一步对文本清洗，过滤多余的空行，让内容更接近原本的段落，章节形式
- 使用UnstructuredMarkdownLoader读取，但会存在很多用不上的metadata属性，所以需要删除过滤

In [151]:
book_name = "comunication_cleaned.md"
book_path = test_data_path / book_name
# 使用unstructured的markdownloader加载
loader = UnstructuredMarkdownLoader(book_path, mode="elements")
documents = loader.load()
# 查看节点数量
print(f"节点数量: {len(documents)}")
# 遍历documents,重构metadata，只保留category，filename这两个属性
for doc in documents:
    category = doc.metadata["category"]
    filename = doc.metadata["filename"]
    doc.metadata = {
        "category": category,
        "filename": filename,
    }


节点数量: 41


In [152]:
from llama_index.core import Document
from llama_index.core.schema import MetadataMode

llamindex_docs = [
    Document(
        text=doc.page_content,
        metadata=doc.metadata,
        metadata_seperator="::",
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    for doc in documents
]
print(
    "The LLM sees this:\n",
    llamindex_docs[0].get_content(metadata_mode=MetadataMode.LLM),
)
print(
    "The Embedding model sees this: \n",
    llamindex_docs[0].get_content(metadata_mode=MetadataMode.EMBED),
)

The LLM sees this:
 Metadata: category=>NarrativeText::filename=>comunication_cleaned.md
-----
Content: 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center

In [62]:
print(len(llamindex_docs))

41


In [18]:
import json

# 查看node有哪些属性,按照json格式输出
print(len(llamindex_docs))
# processed_nodes节点输出全部看看,保存内容到json文件中
with open("llamindex_docs.json", "w", encoding="utf-8") as f:
    for node in llamindex_docs:
        # 数据转json
        json_data = json.dumps(node.to_dict(), ensure_ascii=False, indent=2)
        f.write(json_data)

41


### 使用LLamaindex的MarkdownReader加载,效果不如unstructure

In [153]:
# 使用LLamaindex的MarkdownReader加载
book_name = "comunication_cleaned.md"
book_path = test_data_path / book_name
reader = MarkdownReader()
documents = reader.load_data(book_path)

In [27]:
# print(documents[0])
print(len(documents))
# 输出到txt文件中把所有节点信息
print(documents[0].text)
print(documents[0].metadata)

21
马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。
做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。
非暴力沟通能够：
● 疗愈内心深处的隐秘伤痛；
● 超越个人心智和情感的局限性；
● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式；
● 用不带伤害的方式化解人际间的冲突；
● 学会建立和谐的生命体验。
图书在版编目(CIP)数据
非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5
书名原文：Nonviolent Communication
ISBN 978-7-5222-0051-4
Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49
中国版本图书馆CIP数据核字（2021）第006542号
Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press.
All rights reserved.
Used with permission.
For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org.
版权所有翻印必究
北京市版权局著作权合同登记号：图字01-2016-2253号
非暴力

In [34]:
import json

nodes_list = []
node_category = set()
# 检查所有节点的metadata中一共出现了哪几种category
# 记录过程中最长的内容长度
max_text_len = 0
for doc in llamindex_docs:
    node_category.add(doc.metadata["category"])
    node_data = {
        "text": doc.text,
        "metadata": doc.metadata,
        "text_len": len(doc.text),
    }
    nodes_list.append(node_data)
    if len(doc.text) > max_text_len:
        max_text_len = len(doc.text)
print(node_category)
print(f"最长内容长度: {max_text_len}")

# 保存到 json 文件
# with open("非暴力沟通md.json", "w", encoding="utf-8") as f:
#     json.dump(nodes_list, f, ensure_ascii=False, indent=2)

{'NarrativeText', 'Title', 'UncategorizedText'}
最长内容长度: 13639


可以发现，加载后的段落更完整，更符合直觉的按照章节内容分割成多个节点了，这样可以进一步操作，构建多层级节点，同时也可以去生成一些更好的总结性的文本作为节点内容


### sentence splitter

In [ ]:
from llama_index.core.ingestion import IngestionPipeline

from readai.components.text_splitters.splitters import SentenceSplitter


In [ ]:
# 手动调用SentenceSplitter来查看其行为
ingest_docs = [
    Document(text=doc.page_content, metadata=doc.metadata) for doc in documents
]
# 对一个doc进行分割后查看细节
test_text = ingest_docs[0].text
test_splits = SentenceSplitter.split_text(test_text)
print(f"原文长度: {len(test_text)}, 分割后片段数: {len(test_splits)}")
# 检查原文以及分割后的结果
print(test_text)
print("-" * 100)
for text in test_splits:
    print(text)
    print("-" * 100)

In [ ]:
ingest_docs = [
    Document(text=doc.page_content, metadata=doc.metadata) for doc in documents
]
# 创建SentenceSplitter实例
sentence_splitter = SentenceSplitter(
    chunk_size=512,  # 每个块的最大token数
    chunk_overlap=50,  # 块之间的重叠token数
    include_metadata=True,  # 包含元数据
    include_prev_next_rel=True,  # 维护块之间的前后关系
)

# 直接调用spltter的方式
processed_nodes = sentence_splitter.get_nodes_from_documents(
    ingest_docs, show_progress=True
)


In [13]:
# 查看node有哪些属性,按照json格式输出
print(len(processed_nodes))
# processed_nodes节点输出全部看看,保存内容到json文件中
with open("processed_nodes.json", "w", encoding="utf-8") as f:
    for node in processed_nodes:
        # 数据转json
        json_data = json.dumps(node.to_dict(), ensure_ascii=False, indent=2)
        f.write(json_data)

455


### 对比几种node parser策略

In [47]:
book_nodes = llamindex_docs

In [64]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import IndexNode

node_parser = SimpleNodeParser.from_defaults(
    chunk_size=1024,
    chunk_overlap=50,
    separator="\n\n",
    secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
)
node_parser

SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x3150ac9d0>, id_func=<function default_id_func at 0x10b1283a0>, chunk_size=1024, chunk_overlap=50, separator='\n\n', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')

In [65]:
node_parser.chunk_size
node_parser.chunk_overlap

50

In [66]:
# 采用默认的sentence splitter分块
base_nodes = node_parser.get_nodes_from_documents(llamindex_docs, show_progress=True)


Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

In [67]:
print(len(base_nodes))

170


In [68]:
base_nodes[0].text

'马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org.'

In [61]:
# for idx, node in enumerate(base_nodes):
#     node.id_ = str(idx)
# 不能自己改了，有问题

In [57]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.deepseek import DeepSeek

api_key = os.getenv("DEEPSEEK_API_KEY")
model_name = os.getenv("DEEPSEEK_MODEL")
embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5", base_url="http://localhost:11434"
)

# 设置 LLM
llm = DeepSeek(model=model_name, api_key=api_key)

In [109]:
# 构建index,retriever,query_engine
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(url="http://localhost:6333")
aclient = AsyncQdrantClient(url="http://localhost:6333")

向量初始化

In [113]:
# 如果是传入了nodes类型，直接用类初始化，如果是docs则用from_documents初始化
vector_store_base = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name="base_nodes",
)

collection_name = "base_nodes"
base_index = None
if client.collection_exists(collection_name):
    print(f"集合 '{collection_name}' 已存在")
    base_index = VectorStoreIndex.from_vector_store(
        vector_store_base, embed_model=embed_model
    )
    print("加载本地向量完成")
else:
    print(f"集合 '{collection_name}' 不存在，需要初始化")
    qdrant_storage_context = StorageContext.from_defaults(
        vector_store=vector_store_base
    )
    base_index = VectorStoreIndex(
        base_nodes,
        embed_model=embed_model,
        llm=llm,
        storage_context=qdrant_storage_context,
        show_progress=True,
    )
    print("初始化本地向量完成")

集合 'base_nodes' 已存在
加载本地向量完成


In [71]:
base_retriever = base_index.as_retriever(similarity_top_k=5)

In [72]:
retrievals = base_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)

In [73]:
for n in retrievals:
    display_source_node(n, source_length=1500)

**Node ID:** c5bc47b6-b0d0-4a31-bf2c-cde6f90e2131<br>**Similarity:** 0.65816826<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。 ·我记得你从来没有写信给我。 然而有时，这些词是言过其实的表达，混淆了观察和评论： ·你总是在忙。 ·在需要她的时候，她永远都不在。 这样的表达经常会引发他人的逆反心理，而非慈悲之情。 “经常”“很少”这样的词也可能混淆观察和评论。 小结 非暴力沟通的第一个要素是区分观察与评论。当我们在观察中夹杂着自己的评论时，他人往往会认为我们在批评他们，并因而产生抗拒的心理。<br>

**Node ID:** e87f01ad-2d1b-4eba-989c-7a1003652c07<br>**Similarity:** 0.62561125<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。<br>

**Node ID:** 7c6c297e-5946-4433-975f-c6d5013faa86<br>**Similarity:** 0.62505203<br>**Text:** ” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。 ·我记得你从来没有写信给我。 然而有时，这些词是言过其实的表达，混淆了观察和评论： ·你总是在忙。 ·在需要她的时候，她永远都不在。 这样的表达经常会引发他人的逆反心理，而非慈悲之情。 “经常”“很少”这样的词也可能混淆观察和评论。<br>

**Node ID:** 64e60b97-d59a-4a5a-9d11-9815d748047b<br>**Similarity:** 0.6229984<br>**Text:** 因此，尽管有许多困难，我依然愿意坚持尝试这个方法。 练习一：观察还是评论？ 请完成以下练习，看看你是否可以熟练区分观察和评论。请标出那些只是描述观察而不带有评论的句子。 1. “昨天，约翰无缘无故冲我发脾气。” 2. “昨天晚上，南希一边看电视、一边咬指甲。” 3. “会议上我没有听到桑姆询问我的意见。” 4. “我的父亲是个好人。” 5. “詹尼斯花在工作上的时间太多了。” 6. “亨利很强势。” 7. “这周，潘每天第一个来排队。” 8. “我的儿子经常不刷牙。” 9. “卢克说，我穿黄色衣服不好看。” 10.“姑姑和我说话时一直在抱怨。” 以下是我对练习一的回应： 1.<br>

In [74]:
# 使用retrieverqueryengine
query_engine_base = RetrieverQueryEngine.from_args(base_retriever, llm=llm)

In [77]:
response = query_engine_base.query("在日常交流中，我们如何区分客观的观察和主观的评价？")
print(response)


在日常交流中，区分客观观察和主观评价的关键在于是否包含个人判断或情绪。客观观察应仅描述具体事实或行为，而不附加解释或定性。例如，"南希一边看电视一边咬指甲"是观察，而"詹尼斯花在工作上的时间太多了"则隐含了评价。使用绝对化词语（如"总是""从不"）时需谨慎，这类表达往往将观察夸大成了评论。练习时可以先尝试剥离形容词和价值判断，只保留可验证的行为描述。当我们需要表达感受时，应当基于具体观察展开，而非直接给对方贴标签。这种区分能减少沟通中的对抗性，促进相互理解。


### 采用分级摘要,small2big

In [88]:
sub_chunk_sizes = [128, 512, 2048]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(
        chunk_size=c,
        chunk_overlap=50,
        separator="\n\n",
        secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
    )
    for c in sub_chunk_sizes
]

all_nodes = []
for base_node in base_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([base_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, base_node.node_id) for sn in sub_nodes
        ]
        all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(base_node, base_node.node_id)
    all_nodes.append(original_node)

print(len(all_nodes))

2753


In [79]:
all_nodes_dict = {n.node_id: n for n in all_nodes}

In [80]:
len(all_nodes_dict)

2753

In [114]:
from llama_index.core.retrievers import RecursiveRetriever

collection_name = "small2big_nodes"
vector_store_chunk = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name=collection_name,
)
vector_chunk_index = None

if client.collection_exists(collection_name):
    print(f"集合 '{collection_name}' 已存在")
    vector_chunk_index = VectorStoreIndex.from_vector_store(
        vector_store_chunk, embed_model=embed_model
    )
    print("加载本地向量完成")
else:
    print(f"集合 '{collection_name}' 不存在，需要初始化")
    # build from nodes
    qdrant_storage_context = StorageContext.from_defaults(
        vector_store=vector_store_chunk
    )
    vector_chunk_index = VectorStoreIndex(
        all_nodes,
        embed_model=embed_model,
        storage_context=qdrant_storage_context,
        show_progress=True,
    )


集合 'small2big_nodes' 已存在
加载本地向量完成


In [86]:
vector_retriever_chunk = vector_chunk_index.as_retriever(similarity_top_k=3)

In [87]:
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)

nodes = retriever_chunk.retrieve("在日常交流中，我们如何区分客观的观察和主观的评价？")
for node in nodes:
    display_source_node(node, source_length=2000)

Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: df746b85-3568-48b2-9961-5bea0277c3f6
Retrieving with query id df746b85-3568-48b2-9961-5bea0277c3f6: 在日常交流中，我们如何区分客观的观察和主观的评价？


**Node ID:** df746b85-3568-48b2-9961-5bea0277c3f6<br>**Similarity:** 0.6397337<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

In [89]:
query_engine_chunk = RetrieverQueryEngine.from_args(retriever_chunk, llm=llm)

In [90]:
response = query_engine_chunk.query(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(response)


Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: df746b85-3568-48b2-9961-5bea0277c3f6
Retrieving with query id df746b85-3568-48b2-9961-5bea0277c3f6: 在日常交流中，我们如何区分客观的观察和主观的评价？
在日常交流中，区分客观观察和主观评价的关键在于是否包含个人判断或推测。客观观察应聚焦于具体、可验证的事实，而主观评价则往往带有个人观点或情感色彩。

例如，说"会议超时20分钟"是观察，因为它陈述了可量化的事实；而说"他话太多"则是评价，因为包含了说话者对行为的主观判断。同样，"他讲述了童年经历"是观察，而"他想成为焦点"是评价。

要有效区分两者，可以：
1. 避免使用绝对化词汇（如总是、从不）
2. 描述具体行为而非概括性格
3. 确保陈述的内容可以被摄像机记录
4. 将时间、地点等具体细节纳入描述

这种区分需要练习，因为人们往往习惯于混合表达观察与评价。通过有意识地训练，我们能够更清晰地进行事实性沟通，减少因评价性语言引发的误解或冲突。


### 评估效果

In [92]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
import nest_asyncio

nest_asyncio.apply()

In [94]:
print(len(base_nodes))
print(len(all_nodes))

170
2753


In [98]:
test_nodes = base_nodes[:50]

In [96]:
eval_dataset = generate_question_context_pairs(test_nodes, llm=llm)
eval_dataset.save_json("./feibaoli_eval_dataset50.json")

100%|██████████| 50/50 [07:19<00:00,  8.79s/it]


生成答案测试

In [112]:
import pandas as pd
from llama_index.core.evaluation import RetrieverEvaluator, get_retrieval_results_df

# set vector retriever similarity top k to higher
top_k = 10


def display_results(names, results_arr):
    """Display results from evaluate."""

    hit_rates = []
    mrrs = []
    for name, eval_results in zip(names, results_arr):
        metric_dicts = []
        for eval_result in eval_results:
            metric_dict = eval_result.metric_vals_dict
            metric_dicts.append(metric_dict)
        results_df = pd.DataFrame(metric_dicts)

        hit_rate = results_df["hit_rate"].mean()
        mrr = results_df["mrr"].mean()
        hit_rates.append(hit_rate)
        mrrs.append(mrr)

    final_df = pd.DataFrame({"retrievers": names, "hit_rate": hit_rates, "mrr": mrrs})
    display(final_df)

In [132]:
# base
base_retriever = base_index.as_retriever(similarity_top_k=top_k)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=base_retriever
)

# 异步并行评估需要qdrant aclient
results_base = await retriever_evaluator.aevaluate_dataset(
    eval_dataset, show_progress=True
)

100%|██████████| 100/100 [00:02<00:00, 35.81it/s]


In [ ]:
# chunk
vector_retriever_chunk = vector_chunk_index.as_retriever(similarity_top_k=top_k)
retriever_chunk = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever_chunk},
    node_dict=all_nodes_dict,
    verbose=True,
)
retriever_evaluator_chunk = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever_chunk
)

results_chunk = await retriever_evaluator_chunk.aevaluate_dataset(
    eval_dataset, show_progress=True
)

In [134]:
full_results_df = get_retrieval_results_df(
    ["Base Retriever", "Retriever (Chunk References)"],
    [results_base, results_chunk],
)
display(full_results_df)

,retrievers,hit_rate,mrr
0,Base Retriever,0.29,0.099544
1,Retriever (Chunk References),0.60,0.444119


测试中文问答题目

In [121]:
from readai.components.prompts import QA_GENERATE_PROMPT_TMPL_ZH

eval_dataset_zh = generate_question_context_pairs(
    test_nodes, llm=llm, qa_generate_prompt_tmpl=QA_GENERATE_PROMPT_TMPL_ZH
)

100%|██████████| 50/50 [09:15<00:00, 11.10s/it]


In [125]:
# 重写方法，保存中文内容
import json

path = "./feibaoli_eval_dataset50_zh.json"
with open(path, "w") as f:
    json.dump(eval_dataset_zh.model_dump(), f, indent=4, ensure_ascii=False)

In [135]:
results_base_zh = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)

100%|██████████| 100/100 [00:02<00:00, 33.85it/s]


In [ ]:
results_chunk_zh = await retriever_evaluator_chunk.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)

In [137]:
full_results_df_zh = get_retrieval_results_df(
    ["Base Retriever", "Retriever (Chunk References)"],
    [results_base_zh, results_chunk_zh],
)
display(full_results_df_zh)

,retrievers,hit_rate,mrr
0,Base Retriever,0.32,0.093258
1,Retriever (Chunk References),0.62,0.549000


 ### 探索TextNode

In [ ]:
from llama_index.core.schema import (
    Document,
    MetadataMode,
    NodeRelationship,
    TextNode,
    NodeWithScore,
)

In [ ]:
# 2. 检索阶段
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# 基础向量检索器
vector_retriever = vector_index.as_retriever(similarity_top_k=3)

# 自动合并检索器
auto_retriever = AutoMergingRetriever(
    base_retriever=vector_retriever, storage_context=storage_context
)

# 查询引擎
query_engine = RetrieverQueryEngine.from_args(retriever=auto_retriever)

## 思考
- 为什么要转换成markdown
因为转换成markdown后，原本epub文件是存在一些结构性内容的，比如标题的层级，相比直接按照epub内容（转html）被读取，转换成markdown后，这种层级内容更加清晰，而且我更好处理
- 转换后为什么还要去除一些无效符号？（比如空行）
因为目前很多中文分块策略，在区分段落时会根据两个换行符区分，但转markdown后发现有很多多余的空行,清除掉多余的空行其实是可以更方便使用text_spliiter的

- 要使用哪一种chunk或node parser策略？
    - small2big
    - 句子分割 sentence splitter
- query优化
- 后处理手段
    - rereank